Load and Process Documents

In [ ]:
import os
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import Embedding
import pinecone

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize Pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENVIRONMENT")
)

# Pinecone index name
index_name = "business-qa-bot"

# Load your documents (example: FAQ or business-related content)
documents = [
    "The company's working hours are 9 AM to 5 PM.",
    "We offer a 30-day return policy for all products.",
    "Customer support is available 24/7 via chat and email."
]

# if it is any document like pdf 
#from langchain_community.document_loaders import PyPDFLoader
#loader=PyPDFLoader('attention.pdf')
#docs=loader.load()

# Split documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.create_documents(documents)

# Initialize Pinecone index
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=1536)  # 1536 is the embedding size for OpenAI
index = pinecone.Index(index_name)

Generate Embeddings and Store in Pinecone

In [ ]:
from openai.embeddings_utils import get_embedding

# Generate embeddings for each chunk and upload to Pinecone
for i, chunk in enumerate(chunks):
    embedding = get_embedding(chunk.page_content, engine="text-embedding-ada-002")
    index.upsert([(str(i), embedding, {"text": chunk.page_content})])


Query Processing and Retrieval

In [ ]:
def query_pinecone(query):
    # Generate query embedding
    query_embedding = get_embedding(query, engine="text-embedding-ada-002")

    # Search Pinecone
    results = index.query(query_embedding, top_k=5, include_metadata=True)

    # Return relevant chunks
    return [result['metadata']['text'] for result in results['matches']]


Generate a Response Using OpenAI GPT

In [ ]:
def generate_response(query, retrieved_chunks):
    # Combine the query and retrieved context
    context = "\n".join(retrieved_chunks)
    prompt = f"Answer the following question based on the context below:\n\nContext:\n{context}\n\nQuestion:\n{query}"

    # Generate response
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=200
    )

    return response.choices[0].text.strip()


Main Function for QA Chatbot 

In [ ]:
def chatbot():
    print("Welcome to the Business QA Bot! Type 'exit' to quit.")
    while True:
        query = input("\nYou: ")
        if query.lower() == "exit":
            print("Goodbye!")
            break

        try:
            # Retrieve relevant chunks
            retrieved_chunks = query_pinecone(query)

            # Generate and display response
            response = generate_response(query, retrieved_chunks)
            print(f"\nBot: {response}")
        except Exception as e:
            print(f"\nBot: An error occurred: {e}")


    


In [ ]:
chatbot()